<a href="https://colab.research.google.com/github/takukai0605/machine-learning-FX/blob/master/%E4%BA%88%E6%B8%AC%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21
!git init

/content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21
Initialized empty Git repository in /content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21/.git/


In [ ]:
!git remote rm origin

In [ ]:
!git remote add origin https://github/takukai0605/test2.git

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

#データの入力

In [ ]:
#データの入力
data_0=pd.read_csv('/content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21/USD_JPY 過去データ_4.csv')
data_jp=pd.read_csv("/content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21/日経平均株価 過去データ.csv")
data_ny=pd.read_csv("/content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21/NYダウ平均株価 過去データ.csv")
#data_1=pd.read_csv('/content/drive/MyDrive/個人研究/為替相場予測/FX_2023_04_21/USD_JPY 過去データ_1.csv')
#data_sub = pd.concat([data_0, data_1], axis=0)
#data_sub =data_sub.reset_index(drop=True)

In [ ]:
data_jp["終値"]=data_jp["終値"].str.replace(',','').astype(float)
data_jp["始値"]=data_jp["始値"].str.replace(',','').astype(float)
data_jp["高値"]=data_jp["高値"].str.replace(',','').astype(float)
data_jp["安値"]=data_jp["安値"].str.replace(',','').astype(float)

In [ ]:
data_ny["終値"]=data_ny["終値"].str.replace(',','').astype(float)
data_ny["始値"]=data_ny["始値"].str.replace(',','').astype(float)
data_ny["高値"]=data_ny["高値"].str.replace(',','').astype(float)
data_ny["安値"]=data_ny["安値"].str.replace(',','').astype(float)

In [ ]:
data_sub=pd.merge(data_ny,data_jp,on="日付け")
data_sub=pd.merge(data_sub,data_0,on="日付け")
#data_sub =data_sub.reset_index(drop=True)

#順番を逆にする
data_reverse_0 = data_sub.iloc[::-1]
data_reverse_0.index = range(len(data_sub))

#不必要な行の削除
data_reverse_0=data_reverse_0.drop(['出来高', '変化率 %'], axis=1)

In [ ]:
data_reverse_0.head()

,日付け,終値_x,始値_x,高値_x,安値_x,出来高_x,変化率 %_x,終値_y,始値_y,高値_y,安値_y,出来高_y,変化率 %_y,終値,始値,高値,安値
0,2004-05-10,9990.02,10116.28,10116.28,9932.74,272.13M,-1.26%,10884.70,11384.03,11392.79,10838.93,NaN,-4.84%,113.81,112.26,114.16,112.18
1,2004-05-11,10019.47,9989.24,10038.35,9974.05,223.65M,0.29%,10907.18,10850.48,10970.47,10790.13,NaN,0.21%,112.91,113.88,114.00,112.87
2,2004-05-12,10045.16,10011.52,10048.21,9852.19,246.41M,0.26%,11153.58,11017.65,11157.34,10984.79,NaN,2.26%,113.00,112.94,113.50,112.40
3,2004-05-13,10010.74,10044.31,10074.11,9970.93,215.31M,-0.34%,10825.10,11081.75,11081.75,10825.10,NaN,-2.95%,114.59,112.90,114.66,112.79
4,2004-05-14,10012.87,10008.43,10065.68,9938.42,175.18M,0.02%,10849.63,10846.61,10939.09,10739.49,NaN,0.23%,114.10,114.66,114.93,113.92


#特徴量作成

In [ ]:
#上昇

def up_down(data):
  data['上昇']=0
  for i in range(1,len(data),1):
    j=i-1
    if data['終値'][i]-data['終値'][j]>0:
      data['上昇'][j]=1
    else:
      data['上昇'][j]=0

up_down(data_reverse_0)

<ipython-input-71-48a256177583>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['上昇'][j]=0
<ipython-input-71-48a256177583>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['上昇'][j]=1


In [ ]:
#終値、始値、高値、安値の互いの差分を格納

def difference(data):
  h='高値'
  l='安値'
  s='始値'
  e='終値'
  column=[[h,l],[h,s],[h,e],[l,s],[l,e],[s,e]]
  for i in range(len(column)):
    num1=column[i][0]
    num2=column[i][1]
    data[num1+"-"+num2]=0
    for j in range(len(data)):
      data[num1+"-"+num2][j]=abs(data[num1][j]-data[num2][j])
      
difference(data_reverse_0)

#前日の各値の変化
def before(data):
  h='高値'
  l='安値'
  s='始値'
  e='終値'
  column=[h,l,s,e]
  for i in range(len(column)):
    data["過去"+column[i]+"-"+"現在"+column[i]]=0
    for j in range(1,len(data),1):
      k=j-1
      data["過去"+column[i]+"-"+"現在"+column[i]][j]=data[column[i]][k]-data[column[i]][j]
      
before(data_reverse_0)

def difference2(data):
  h='高値'
  l='安値'
  s='始値'
  e='終値'
  column=[[h,l],[h,s],[h,e],[l,s],[l,e],[s,e]]
  for i in range(len(column)):
    num1=column[i][0]
    num2=column[i][1]
    data[num1+"-"+num2+"-2"]=0
    for j in range(len(data)):
      data[num1+"-"+num2+"-2"][j+1]=abs(data[num1][j]-data[num2][j])
      
difference2(data_reverse_0)

def difference3(data):
  h='高値'
  l='安値'
  s='始値'
  e='終値'
  column=[[h,l],[h,s],[h,e],[l,s],[l,e],[s,e]]
  for i in range(len(column)):
    num1=column[i][0]
    num2=column[i][1]
    data[num1+"-"+num2+"-3"]=0
    for j in range(len(data)):
      data[num1+"-"+num2+"-3"][j+2]=abs(data[num1][j]-data[num2][j])
      
difference3(data_reverse_0)

<ipython-input-72-3f60cc61b28a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[num1+"-"+num2][j]=abs(data[num1][j]-data[num2][j])
<ipython-input-72-3f60cc61b28a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[num1+"-"+num2][j]=abs(data[num1][j]-data[num2][j])
<ipython-input-72-3f60cc61b28a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[num1+"-"+num2][j]=abs(data[num1][j]-data[num2][j])
<ipython-input-72-3f60cc6

In [ ]:
data_jp.head()

,日付け,終値,始値,高値,安値,出来高,変化率 %
0,2023-05-10,29122.18,29189.41,29195.16,29070.11,872.76M,-0.41%
1,2023-05-09,29242.82,29020.63,29262.36,29020.63,863.91M,1.01%
2,2023-05-08,28949.88,29095.46,29144.12,28931.81,741.49M,-0.71%
3,2023-05-02,29157.95,29278.80,29278.80,29083.13,639.10M,0.12%
4,2023-05-01,29123.18,29058.05,29145.89,29016.83,733.68M,0.92%


In [ ]:
#データ分割
from sklearn.model_selection import train_test_split
data_reverse, data_reverse_test = train_test_split(data_reverse_0, test_size=0.3,shuffle=False)

data_reverse_test=data_reverse_test.reset_index()

In [ ]:
column=['高値-安値', '高値-始値','高値-終値', '安値-始値', '安値-終値', '始値-終値'
  ,'過去高値-現在高値','過去安値-現在安値', '過去始値-現在始値', '過去終値-現在終値']
top= 200
list=np.zeros((len(column),top))
for j in range(top):
  for i in range(len(column)):
    list[i][j]=round(data_reverse[column[i]][data_reverse["上昇"]==1].value_counts().head(top).index[j],2)

for k in range(len(column)):
  data_reverse[column[k]+'-count']=0
  for j in range(top):
    for i in range(len(data_reverse)):
      if round(data_reverse[column[k]][i],2)==list[k][j]:
        data_reverse[column[k]+'-count'][i]=top-j

for k in range(len(column)):
  data_reverse_test[column[k]+'-count']=0
  for j in range(top):
    for i in range(len(data_reverse_test)):
      if round(data_reverse_test[column[k]][i],2)==list[k][j]:
        data_reverse_test[column[k]+'-count'][i]=top-j

<ipython-input-76-2e0c8bd9f3ad>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reverse[column[k]+'-count'][i]=top-j
<ipython-input-76-2e0c8bd9f3ad>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reverse[column[k]+'-count'][i]=top-j
<ipython-input-76-2e0c8bd9f3ad>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reverse[column[k]+'-count'][i]=top-j
<ipython-input-76-2e0c8bd9f3ad>:14: SettingWithCopyWarning: 
A valu

In [ ]:
data_reverse.columns

Index(['日付け', '終値_x', '始値_x', '高値_x', '安値_x', '出来高_x', '変化率 %_x', '終値_y',
       '始値_y', '高値_y', '安値_y', '出来高_y', '変化率 %_y', '終値', '始値', '高値', '安値',
       '上昇', '高値-安値', '高値-始値', '高値-終値', '安値-始値', '安値-終値', '始値-終値', '過去高値-現在高値',
       '過去安値-現在安値', '過去始値-現在始値', '過去終値-現在終値', '高値-安値-2', '高値-始値-2', '高値-終値-2',
       '安値-始値-2', '安値-終値-2', '始値-終値-2', '高値-安値-3', '高値-始値-3', '高値-終値-3',
       '安値-始値-3', '安値-終値-3', '始値-終値-3', '高値-安値-count', '高値-始値-count',
       '高値-終値-count', '安値-始値-count', '安値-終値-count', '始値-終値-count',
       '過去高値-現在高値-count', '過去安値-現在安値-count', '過去始値-現在始値-count',
       '過去終値-現在終値-count'],
      dtype='object')

In [ ]:
x_train=data_reverse[['高値-安値-count', '高値-始値-count',
       '高値-終値-count', '安値-始値-count', '安値-終値-count', '始値-終値-count',
       '過去高値-現在高値-count', '過去安値-現在安値-count', '過去始値-現在始値-count',
       '過去終値-現在終値-count', '終値', '始値', '高値', '安値', '終値_y',
       '始値_y', '高値_y', '安値_y', '終値_x', '始値_x', '高値_x', '安値_x']].values
y_train=data_reverse[['上昇']].values

x_test=data_reverse_test[['高値-安値-count', '高値-始値-count',
       '高値-終値-count', '安値-始値-count', '安値-終値-count', '始値-終値-count',
       '過去高値-現在高値-count', '過去安値-現在安値-count', '過去始値-現在始値-count',
       '過去終値-現在終値-count', '終値', '始値', '高値', '安値', '終値_y',
       '始値_y', '高値_y', '安値_y', '終値_x', '始値_x', '高値_x', '安値_x']].values
y_test=data_reverse_test[['上昇']].values

#モデル学習

In [ ]:
#GBDT
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier(max_depth=7)
gbdt.fit(x_train,y_train)
pred_gbdt_train=gbdt.predict(x_train)
pred_gbdt_test=gbdt.predict(x_test)

print('正答率 '+str(round(accuracy_score(y_test, pred_gbdt_test)*100,2))+'％')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


正答率 50.67％


In [ ]:
#多層ニューラルネットワーク
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=1000)
mlp.fit(x_train,y_train)
pred_mlp_train=mlp.predict(x_train)
pred_mlp_test=mlp.predict(x_test)

print (accuracy_score(y_test, pred_mlp_test))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.48518518518518516


#売買シグナル

In [ ]:
#売買シグナル（1なら買い、0なら売り）
pred_gbdt_test[-2]

0

In [ ]:
!git add .

In [ ]:
!git config --global user.email "takukai0605@gmail.com"
!git config --global user.name "kaitakuma"

In [ ]:
!git commit -m"update"

[master (root-commit) 4fa5d4f] update
 10 files changed, 32720 insertions(+)
 create mode 100644 "NY\343\202\277\343\202\231\343\202\246\345\271\263\345\235\207\346\240\252\344\276\241 \351\201\216\345\216\273\343\203\206\343\202\231\343\203\274\343\202\277.csv"
 create mode 100644 "USD_JPY \351\201\216\345\216\273\343\203\206\343\202\231\343\203\274\343\202\277_0.csv"
 create mode 100644 "USD_JPY \351\201\216\345\216\273\343\203\206\343\202\231\343\203\274\343\202\277_1.csv"
 create mode 100644 "USD_JPY \351\201\216\345\216\273\343\203\206\343\202\231\343\203\274\343\202\277_2.csv"
 create mode 100644 "USD_JPY \351\201\216\345\216\273\343\203\206\343\202\231\343\203\274\343\202\277_3.csv"
 create mode 100644 "USD_JPY \351\201\216\345\216\273\343\203\206\343\202\231\343\203\274\343\202\277_4.csv"
 create mode 100644 "\344\272\210\346\270\254\343\203\242\343\203\206\343\202\231\343\203\253.ipynb"
 create mode 100644 "\344\272\210\346\270\254\343\203\242\343\203\206\343\202\231\343\203\2

In [ ]:
% ssh-keygen -t rsa

UsageError: Line magic function `%` not found.


In [ ]:
!git branch

* master


In [ ]:
!git remote rm origin

In [ ]:
!git remote add origin https://github.com/takukai0605/test2.git

In [ ]:
!git push origin master 

fatal: unable to access 'https://github.com/takukai0605/test2.git/': Received HTTP code 400 from proxy after CONNECT
